In [7]:
import glob
import numpy as np
import tensorflow as tf
from scipy.io import wavfile

In [8]:
clean_wav_files = glob.glob('../data/noisy_trainset_28spk_wav/*.wav')
noisy_wav_files = glob.glob('../data/noisy_trainset_28spk_wav/*.wav')

clean_data = [wavfile.read(file)[1] for file in clean_wav_files]
noisy_data = [wavfile.read(file)[1] for file in noisy_wav_files]
frame_rate = wavfile.read(clean_wav_files[0])[0]

In [9]:
sequence_length = int(frame_rate*40/1000) # 40 miliseconds time windows
sequence_shift = int(frame_rate*16/1000)  # 16 milisceonds shifts
print(frame_rate, sequence_length, sequence_shift)

48000 1920 768


In [10]:
def window(noisy_data: np.array, clean_data: np.array, shift, length, limit=-1):
    """[summary]

    Args:
        noisy_data (np.array): [description]
        clean_data (np.array): [description]

    Returns:
        [type]: [description]
    """
    x = []
    y = []
    for i in range(len(noisy_data)):
        for j in range((len(noisy_data[i]) - length)//shift + 1):
            x.append(noisy_data[i][j*shift: j*shift + length])
            y.append(clean_data[i][j*shift: j*shift + length])
        yield np.expand_dims(np.array(x), axis=0), np.expand_dims(np.array(y), axis=0)

In [6]:
train_generator = window(noisy_data,
                         clean_data,
                         shift=sequence_shift,
                         length=sequence_length,
                         limit=10)

for i in train_generator:
    print(i[0].shape, i[1].shape)

NameError: name 'window' is not defined

In [6]:
import matplotlib.pyplot as plt

# scale the whole data and the windowed them!
# aggregation -> what happens when we have overlap -> should we weight them
# the two prediction might have overlaps -> look at the standard deviation of the overlaps!!

plt.plot(x_train[0][0].tolist())

NameError: name 'x_train' is not defined

In [2]:
def GRU_model(sequence_length, neurons=40):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, GRU
    
    model = Sequential()
    model.add(GRU(neurons,
                  return_sequences=False,
                  return_state=False,
                  input_shape=(1, sequence_length)))
    model.add(Dense(sequence_length, activation='relu'))
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['acc'])
    
    return model

In [1]:
def LSTM_model(sequence_length, neurons=40):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LSTM
    
    model = Sequential()
    model.add(LSTM(neurons,
                   return_sequences=False,
                   return_state=False,
                   input_shape=(1, sequence_length))),
    model.add(Dense(sequence_length, activation='relu'))
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['acc'])
    
    return model

In [12]:
model = LSTM_model(sequence_length, neurons=10)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 10)                57960     
_________________________________________________________________
dense (Dense)                (None, 1920)              21120     
Total params: 79,080
Trainable params: 79,080
Non-trainable params: 0
_________________________________________________________________


Epoch 1/1000


InvalidArgumentError:  Can not squeeze dim[2], expected a dimension of 1, got 1920
	 [[node mean_squared_error/remove_squeezable_dimensions/Squeeze (defined at <ipython-input-13-31dbb08eebd0>:1) ]] [Op:__inference_train_function_2967]

Function call stack:
train_function
